In [1]:
import pandas as pd
import plotly.graph_objects as go
import datetime as dt
from plotting import CandlePlot

In [2]:
pair = "EUR_USD"
granularity = "H4"
df = pd.read_pickle(f"../data/{pair}_{granularity}.pkl")
MA_LIST = [10, 20, 50, 100, 200]

In [3]:
df.head()

,time,volumne,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
0,2022-06-09 05:00:00+00:00,22278,1.07266,1.07296,1.06940,1.07155,1.07258,1.07288,1.06932,1.07147,1.07273,1.07303,1.06947,1.07163
1,2022-06-09 09:00:00+00:00,73673,1.07156,1.07742,1.06882,1.07344,1.07148,1.07733,1.06865,1.07335,1.07163,1.07750,1.06899,1.07353
2,2022-06-09 13:00:00+00:00,68304,1.07344,1.07432,1.06452,1.06524,1.07334,1.07423,1.06444,1.06516,1.07354,1.07442,1.06459,1.06532
3,2022-06-09 17:00:00+00:00,21354,1.06526,1.06550,1.06113,1.06188,1.06517,1.06543,1.06104,1.06179,1.06534,1.06557,1.06121,1.06196
4,2022-06-09 21:00:00+00:00,12387,1.06172,1.06246,1.06112,1.06241,1.06141,1.06237,1.06102,1.06233,1.06203,1.06254,1.06121,1.06249


In [4]:
df.columns

Index(['time', 'volumne', 'mid_o', 'mid_h', 'mid_l', 'mid_c', 'bid_o', 'bid_h',
       'bid_l', 'bid_c', 'ask_o', 'ask_h', 'ask_l', 'ask_c'],
      dtype='object')

In [5]:
df_ma = df[['time','mid_o', 'mid_h', 'mid_l', 'mid_c']].copy()

In [8]:
df_ma.head(10)

,time,mid_o,mid_h,mid_l,mid_c,MA_10,MA_20,MA_50,MA_100,MA_200
0,2022-07-26 09:00:00+00:00,1.02081,1.02082,1.01162,1.01484,1.021136,1.020775,1.016367,1.017225,1.034101
1,2022-07-26 13:00:00+00:00,1.01482,1.01530,1.01076,1.01272,1.020584,1.020455,1.016581,1.016918,1.033807
2,2022-07-26 17:00:00+00:00,1.01268,1.01328,1.01140,1.01174,1.019684,1.019716,1.016812,1.016607,1.033498
3,2022-07-26 21:00:00+00:00,1.01164,1.01442,1.01164,1.01404,1.018866,1.019512,1.017111,1.016310,1.033242
4,2022-07-27 01:00:00+00:00,1.01400,1.01522,1.01369,1.01508,1.017918,1.019114,1.017372,1.016010,1.033008
5,2022-07-27 05:00:00+00:00,1.01508,1.01586,1.01276,1.01418,1.017122,1.018799,1.017617,1.015728,1.032767
6,2022-07-27 09:00:00+00:00,1.01416,1.01720,1.01384,1.01430,1.016328,1.018609,1.017842,1.015449,1.032523
7,2022-07-27 13:00:00+00:00,1.01431,1.01524,1.00968,1.01110,1.015136,1.018398,1.018001,1.015126,1.032272
8,2022-07-27 17:00:00+00:00,1.01112,1.02209,1.01066,1.01996,1.014876,1.018460,1.018256,1.014887,1.032110
9,2022-07-27 21:00:00+00:00,1.01990,1.02124,1.01804,1.01864,1.014660,1.018231,1.018466,1.014766,1.031945


In [7]:
for ma in MA_LIST:
    df_ma[f'MA_{ma}'] = df_ma.mid_c.rolling(window=ma).mean()
df_ma.dropna(inplace=True)
df_ma.reset_index(inplace=True, drop=True)

In [10]:
df_plot = df_ma.iloc[:500]

In [11]:
cp = CandlePlot(df_plot)

In [13]:
traces = [ f"MA_{x}" for x in MA_LIST ]

In [14]:
cp.show_plot(line_traces=traces)